# Set up the necessities and self-written functions

In [ ]:
source("R_functions_forJupyter.R")

# Reading in the Data

In [ ]:
data<-read.csv("Whiz_Memory_Full_Data_2018.csv")
data$pid<-as.factor(data$pid)
data$group<-as.factor(data$group)

# exclude children with IQ > 140, or < 85

In [ ]:
data_match<-data[(data$fsiq >85 & data$fsiq<140),]

# exclude the RRIB part to create the group comparison t table

In [ ]:
data_match1<-data_match[,-c(grep("RRIB_",colnames(data_match)),grep("ADI_",colnames(data_match)),grep("ADOS_",colnames(data_match)))] 
mk_ttable(data_match1,2,6,"BPSO_ASD_TD_pre_Match_sample_t-test.csv",2,3,equalvar=T)

# descriptives for two groups
## for ASD group

In [ ]:
mk_destable(data_match[data_match$group=="ASD",],2,6,"BPSO",2)

## for TD group

In [ ]:
mk_destable(data_match[data_match$group=="TD",-c(grep("RRIB_",colnames(data_match)),
            grep("ADI_",colnames(data_match)),grep("ADOS_",colnames(data_match)))],2,6,"BPSO",2)

# calculate summary metrics from the raw data in BPSO

## acc based on types (corrected)

In [ ]:
data_match$OldAcc<-data_match$old_given_target_count/data_match$tot_target
data_match$Old_lure_err<-data_match$sim_given_target_count/data_match$tot_target
data_match$Old_new_err<-data_match$new_given_target_count/data_match$tot_target
data_match$LureAcc<-data_match$sim_given_lure_count/data_match$tot_lure
data_match$Lure_old_err<-data_match$old_given_lure_count/data_match$tot_lure
data_match$Lure_new_err<-data_match$new_given_lure_count/data_match$tot_lure
data_match$NewAcc<-data_match$new_given_foil_count/data_match$tot_foil
data_match$New_old_err<-data_match$old_given_foil_count/data_match$tot_foil
data_match$New_lure_err<-data_match$sim_given_foil_count/data_match$tot_foil

## traditional recognition memory: discrimination between OLD and NEW

In [ ]:
data_match$dprime_old_new<-(qnorm(ifelse(data_match$old_given_target_corr==0,
                                         data_match$old_given_target_corr+0.001,data_match$old_given_target_corr-0.001))
                            -qnorm(data_match$old_given_foil_corr+0.001))

## new measure for pattern separation: discrimination bewteen OLD and SIM

In [ ]:
data_match$dprime_old_lure<-(qnorm(ifelse(data_match$old_given_target_corr==0,
                                          data_match$old_given_target_corr+0.001,data_match$old_given_target_corr-0.001))
                             -qnorm(data_match$old_given_lure_corr+0.001))

## pattern separation measure (1 - p(old)) at repitition, L1-L5, new for a tuning curve

In [ ]:
data_match$Sep_Rep<-100-data_match$old_given_target_corr*100
data_match$Sep_L1<-100-data_match$L1O/(data_match$L1O+data_match$L1S+data_match$L1N)*100
data_match$Sep_L2<-100-data_match$L2O/(data_match$L2O+data_match$L2S+data_match$L2N)*100
data_match$Sep_L3<-100-data_match$L3O/(data_match$L3O+data_match$L3S+data_match$L3N)*100
data_match$Sep_L4<-100-data_match$L4O/(data_match$L4O+data_match$L4S+data_match$L4N)*100
data_match$Sep_L5<-100-data_match$L5O/(data_match$L5O+data_match$L5S+data_match$L5N)*100
data_match$Sep_New<-100-data_match$old_given_foil_corr*100

# clustering
## for ASD

In [ ]:
require(NbClust)
data_ASD<-with(data_match[data_match$group=="ASD",],cbind(Sep_Rep,Sep_L1,Sep_L2,Sep_L3,Sep_L4,Sep_L5,Sep_New))
clust.ASD<-NbClust(data=data_ASD,distance="euclidean",min.nc=2,max.nc=8,method="ward.D2",index="alllong")

## for TD

In [ ]:
data_TD<-with(data_match[data_match$group=="TD",],cbind(Sep_Rep,Sep_L1,Sep_L2,Sep_L3,Sep_L4,Sep_L5,Sep_New))
clust.TD<-NbClust(data=data_TD,distance="euclidean",min.nc=2,max.nc=8,method="ward.D2",index="alllong")

## Assigning clustering membership to the data

In [ ]:
data_match$hclust<-as.factor(c(clust.ASD$Best.partition,clust.TD$Best.partition))
data_match<-data_match[,c(1:5,ncol(data_match),6:(ncol(data_match)-1))]
data_match$hclust<-as.factor(data_match$hclust)

# create graphs to visualize the tuning curves
## preparing the data

In [ ]:
data_part<-data_match[,which(colnames(data_match) %in% c("pid","group","Sep_Rep","Sep_L1","Sep_L2","Sep_L3","Sep_L4","Sep_L5","Sep_New","hclust"))]
data_part_long<-melt(data_part,id.vars=c("pid","group","hclust"))
data_long_sum<-summarySE(data_part_long,measurevar="value",groupvar=c("group","hclust","variable"))
print(data_long_sum)

## for ASD group

In [ ]:
line_graph_ASD<-ggplot(data_long_sum[data_long_sum$group=="ASD",],aes(x=variable,y=value,group=hclust,colour=hclust)) +
 			geom_line() +
 			geom_point(size=3) +
 			geom_errorbar(aes(ymin=value-se,ymax=value+se),width=0.4) +
 			scale_colour_manual(values=c("orangered","blue","green")) +
 			scale_x_discrete(labels=c("Rep","L1","L2","L3","L4","L5","New")) +
 			labs(y = "1 - p (OLD)", x = "Stimuli type") +
 			theme_bw()

print(line_graph_ASD)
pdf(file="BPSO_ASD_subgroup_Match_sample_Criterion_report_OLD_curves.pdf",width=8,height=8)
print(line_graph_ASD)
dev.off()

## for TD group

In [ ]:
line_graph_TD<-ggplot(data_long_sum[data_long_sum$group=="TD",],aes(x=variable,y=value,group=hclust,colour=hclust)) +
  geom_line() +
  geom_point(size=3) +
  geom_errorbar(aes(ymin=value-se,ymax=value+se),width=0.4) +
  scale_colour_manual(values=c("orangered","blue","green")) +
  scale_x_discrete(labels=c("Rep","L1","L2","L3","L4","L5","New")) +
  labs(y = "1 - p (OLD)", x = "Stimuli type") +
  theme_bw()

print(line_graph_TD)
pdf(file="BPSO_TD_subgroup_Match_sample_Criterion_report_OLD_curves.pdf",width=8,height=8)
print(line_graph_TD)
dev.off()

# t-tests between ASD and TD, as well as t-tests between sub-groups
## between ASD and TD

In [ ]:
mk_ttable(data_match[,-c(grep("RRIB_",colnames(data_match)),grep("ADI_",colnames(data_match)),grep("ADOS_",colnames(data_match)))],2,7,"BPSO_ASD_TD_Match_sample_t-test.csv",2,3,equalvar=T)

## between ASD subgroup 1 vs 3

In [ ]:
mk_ttable(data_match[(data_match$group=="ASD" & data_match$hclust!=2),],6,7,"BPSO_ASD_subgroup1vs3_t-test_Porb_of_OLD.csv",2,3,equalvar=T)

## between ASD subgroup 2 vs 3

In [ ]:
mk_ttable(data_match[(data_match$group=="ASD" & data_match$hclust!=1),],6,7,"BPSO_ASD_subgroup2vs3_t-test_Porb_of_OLD.csv",2,3,equalvar=T)

## between ASD subgroup 1 vs 2

In [ ]:
mk_ttable(data_match[(data_match$group=="ASD" & data_match$hclust!=3),],6,7,"BPSO_ASD_subgroup1vs2_t-test_Porb_of_OLD.csv",2,3,equalvar=T)

## between TD subgroup 1 vs 2

In [ ]:
mk_ttable(data_match[data_match$group=="TD",-c(grep("RRIB_",colnames(data_match)),grep("ADI_",colnames(data_match)),grep("ADOS_",colnames(data_match)))],6,7,"BPSO_TD_subgroup1vs2_t-test_Porb_of_OLD.csv",2,3,equalvar=T)

# correlations within ASD and TD, as well as within each sub-group
## within ASD

In [ ]:
mk_rtable(data_match[data_match$group=="ASD",],7,2,"BPSO_ASD_match_group")

## within ASD Subgroup 1 (impaired)

In [ ]:
mk_rtable(data_match[(data_match$group=="ASD" & data_match$hclust==1),],7,2,"BPSO_ASD_match_subgroup1")

## within ASD Subgroup 2 (weak)

In [ ]:
mk_rtable(data_match[(data_match$group=="ASD" & data_match$hclust==2),],7,2,"BPSO_ASD_match_subgroup2")

## within ASD Subgroup 3 (super)

In [ ]:
mk_rtable(data_match[(data_match$group=="ASD" & data_match$hclust==3),],7,2,"BPSO_ASD_match_subgroup3")

## within TD

In [ ]:
mk_rtable(data_match[data_match$group=="TD",-c(grep("RRIB_",colnames(data_match)),grep("ADI_",colnames(data_match)),grep("ADOS_",colnames(data_match)))],7,2,"BPSO_TD_match_group")

# make bar graph between ASD and TD
## overall acc

In [ ]:
data_temp<-data_match[,which(colnames(data_match) %in% c("pid","group","percent_corr","OldAcc","LureAcc","NewAcc"))]
data_temp$percent_corr<-data_temp$percent_corr/100
mk_grpbargraph(data_temp,2,3:ncol(data_temp),c("ASD","TD"),"ASD_TD_Overall_Acc_comp",0)

## d-prime

In [ ]:
data_temp<-data_match[,which(colnames(data_match) %in% c("pid","group","dprime_old_new","dprime_old_lure","dprime_lure_new"))]
mk_grpbargraph(data_temp,2,3:ncol(data_temp),c("ASD","TD"),"ASD_TD_d-prime_comp",0)

## WM

In [ ]:
data_temp<-data_match[,which(colnames(data_match) %in% c("pid","group","digit_recall","backwards_digit_recall","block_recall","spatial_recall"))]
mk_grpbargraph(data_temp,2,3:ncol(data_temp),c("ASD","TD"),"ASD_TD_WM_comp",0)

In [ ]:
## Visualspatial Memory

In [ ]:
data_temp<-data_match[,which(colnames(data_match) %in% c("pid","group","MD_Spatial_Scaled_Score","MD_Content_Scaled_Score","MDD_Spatial_Scaled_Score","MDD_Content_Scaled_Score"))]
mk_grpbargraph(data_temp,2,3:ncol(data_temp),c("ASD","TD"),"ASD_TD_VisuospatialWM_comp",0)

# make bar graph of subclusters
## age (ASD and TD)

In [ ]:
data_temp<-data_match[,which(colnames(data_match) %in% c("pid","group","hclust","age"))]
data_temp_sum<-summarySE(data_temp,measurevar = "age",groupvar=c("group","hclust"),na.rm=T)
bar_graph<-ggplot(data_temp_sum,aes(x=group,y=age,fill=factor(hclust))) +
           geom_bar(position=position_dodge(), stat="identity",color="black", width=0.8) +               
           geom_errorbar(aes(ymin=age-se,ymax=age+se),width=0.4,position=position_dodge(.8)) +
           coord_cartesian(ylim=c(min(data_temp_sum$age)-max(data_temp_sum$sd)/2,max(data_temp_sum$age)+max(data_temp_sum$sd)/2)) +
           scale_fill_manual(values=c("orangered","blue","green")) +
           labs(y="age",x="group") +
           theme_bw()
pdf(paste0("ASD_TD","_","age_cluster_group_comp_bargraph.pdf"),height=5,width=5)
plot(bar_graph)
dev.off()
print(bar_graph)

## IQ Measures (ASD)

In [ ]:
data_temp<-data_match[,which(colnames(data_match) %in% c("pid","group","hclust","viq","piq","fsiq"))]
mk_grpbargraph(data_temp[data_temp$group=="ASD",],3,4:ncol(data_temp),c("1","2","3"),"ASD_IQ_subcluster",0)

## IQ Meausres (TD)

In [ ]:
mk_grpbargraph(data_temp[data_temp$group=="TD",],3,4:ncol(data_temp),c("1","2"),"TD_IQ_subcluster",0)

## performance Acc (ASD and TD)

In [ ]:
data_temp<-data_match[,which(colnames(data_match) %in% c("pid","group","hclust","percent_corr"))]
data_temp_sum<-summarySE(data_temp,measurevar = "percent_corr",groupvar=c("group","hclust"),na.rm=T)
bar_graph<-ggplot(data_temp_sum,aes(x=group,y=percent_corr,fill=factor(hclust))) +
  geom_bar(position=position_dodge(), stat="identity",color="black", width=0.8) +               
  geom_errorbar(aes(ymin=percent_corr-se,ymax=percent_corr+se),width=0.4,position=position_dodge(.8)) +
  coord_cartesian(ylim=c(min(data_temp_sum$percent_corr)-max(data_temp_sum$sd)/2,max(data_temp_sum$percent_corr)+max(data_temp_sum$sd)/2)) +
  scale_fill_manual(values=c("orangered","blue","green")) +
  labs(y="Overall Accuracy (corrected)",x="group") +
  theme_bw()
pdf(paste0("ASD_TD","_","OverallCorrectedACC_cluster_group_comp_bargraph.pdf"),height=5,width=5)
plot(bar_graph)
dev.off()
plot(bar_graph)

## dprime (ASD)

In [ ]:
data_temp<-data_match[,which(colnames(data_match) %in% c("pid","group","hclust","dprime_old_new","dprime_old_lure"))]
mk_grpbargraph(data_temp[data_temp$group=="ASD",],3,4:ncol(data_temp),c("1","2","3"),"ASD_dprime1_subcluster",0)

## dprime (TD)

In [ ]:
mk_grpbargraph(data_temp[data_temp$group=="TD",],3,4:ncol(data_temp),c("1","2"),"TD_dprime1_subcluster",0)

## WM (ASD)

In [ ]:
data_temp<-data_match[,which(colnames(data_match) %in% c("pid","group","hclust","digit_recall","backwards_digit_recall","block_recall","spatial_recall"))]
mk_grpbargraph(data_temp[data_temp$group=="ASD",],3,4:ncol(data_temp),c("1","2","3"),"ASD_WM_subcluster",0)

## WM (TD)

In [ ]:
mk_grpbargraph(data_temp[data_temp$group=="TD",],3,4:ncol(data_temp),c("1","2"),"TD_WM_subcluster",0)

## Visuospatial memory for design (NEPSY; ASD)

In [ ]:
data_temp<-data_match[,which(colnames(data_match) %in% c("pid","group","hclust","MD_Spatial_Scaled_Score","MD_Content_Scaled_Score","MDD_Spatial_Scaled_Score","MDD_Content_Scaled_Score"))]
mk_grpbargraph(data_temp[data_temp$group=="ASD",],3,4:ncol(data_temp),c("1","2","3"),"ASD_NEPSY_MD_subcluster",0)

## Visuospatial memory for design (NEPSY; TD)

In [ ]:
mk_grpbargraph(data_temp[data_temp$group=="TD",],3,4:ncol(data_temp),c("1","2"),"TD_NEPSY_MD_subcluster",0)

## Memory for design and faces (NEPSY; ASD)

In [ ]:
data_temp<-data_match[,which(colnames(data_match) %in% c("pid","group","hclust","MD_Scaled_Score","MDD_Total_Scaled_Score","MF_Scaled_Score","MFD_Scaled_Score"))]
mk_grpbargraph(data_temp[data_temp$group=="ASD",],3,4:ncol(data_temp),c("1","2","3"),"ASD_NEPSY_MDF_total_subcluster",0)

## Memory for design and faces (NEPSY; TD)

In [ ]:
mk_grpbargraph(data_temp[data_temp$group=="TD",],3,4:ncol(data_temp),c("1","2"),"TD_NEPSY_MDF_total_subcluster",0)

In [ ]:
## ASD symptom measures

## RRIB

In [ ]:
data_temp<-data_match[,c(which(colnames(data_match) %in% c("pid","group","hclust")),grep("RRIB_",colnames(data_match)))]
mk_grpbargraph(data_temp[data_temp$group=="ASD",],3,4:ncol(data_temp),c("1","2","3"),"ASD_RRIB_subcluster",0)

## ADI

In [ ]:
data_temp<-data_match[,c(which(colnames(data_match) %in% c("pid","group","hclust")),grep("ADI_",colnames(data_match)))]
mk_grpbargraph(data_temp[data_temp$group=="ASD",],3,4:ncol(data_temp),c("1","2","3"),"ASD_ADI_subcluster",0)

## ADOS

In [ ]:
data_temp<-data_match[,c(which(colnames(data_match) %in% c("pid","group","hclust")),grep("ADOS_",colnames(data_match)))]
mk_grpbargraph(data_temp[data_temp$group=="ASD",],3,4:ncol(data_temp),c("1","2","3"),"ASD_ADOS_subcluster",0)